In [2]:
from sklearn.linear_model import LogisticRegression

class FeatureSelection:
    
    def forward_feature_selection(self, X, y, n_features=2):
        results = {'best':[]}
        feature_list = [] #
        while len(feature_list) < n_features:
            best = None
            for i in range(X.shape[1]):
                if i not in feature_list: 
                    cur_model_result = train_model(X, y, feature_list + [i])
                    best = get_best(best, cur_model_result)
            feature_list = best['features']
            results['best'].append(best)
        self.result = compute_best_features(results, n_features)
        return results

    def backward_feature_selection(self, X, y, n_features=2):
        results = {'best':[]}
        feature_list = range(X.shape[1])
        while len(feature_list) > 1:
            best = None
            for i in range(len(feature_list)):
                idx = feature_list[0:i] + feature_list[i+1:]
                best = get_best(best, train_model(X, y, idx))
            feature_list = best['features']
            results['best'].append(best)
        self.result =  compute_best_features(results, n_features)
        return results

def compute_best_features(results, n_features):
    best_result = None
    for result in results['best']:
        if len(result['features']) <= n_features:
            best_result = get_best(best_result, result)
    results['best_features'] = best_result['features']
    
def train_model(X, y, idx):
    X_ = X[:, idx]
    model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_, y)
    acc = accuracy_score(y, model.predict(X_))
    return {'score': acc, 'features': idx}

def get_best(b1, b2):
    return b2 if b1 == None or b2['score'] > b1['score'] else b1

In [3]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data 
Y = iris.target
print('**********Forward*********')
print(FeatureSelection().forward_feature_selection(X, Y))
print('\n')
print('**********Backward*********')
print(FeatureSelection().backward_feature_selection(X, Y))

**********Forward*********
{'best_features': [3, 2], 'best': [{'score': 0.96, 'features': [3]}, {'score': 0.9666666666666667, 'features': [3, 2]}]}


**********Backward*********
{'best_features': [2, 3], 'best': [{'score': 0.9733333333333334, 'features': [1, 2, 3]}, {'score': 0.9666666666666667, 'features': [2, 3]}, {'score': 0.96, 'features': [3]}]}
